In [ ]:
import os
os.chdir("../")
%pwd

In [ ]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [ ]:
import os
import sys
#sys.path.append(os.path.join(os.path.dirname(__file__), ".."))

from MLOPS.src.mlops.constants import *
from MLOPS.src.mlops.utils.common import read_yaml, create_directories

class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )
        return data_ingestion_config

In [ ]:
import os 
import urllib.request as request
import zipfile
from MLOPS.src.mlops.utils.common import get_size
from MLOPS.src.mlops import logger

class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename = os.path.basename(self.config.source_URL)
            target_file = os.path.join(self.config.unzip_dir, filename)
            logger.info(f"Downloading file from : {self.config.source_URL} to {target_file}")
            if not os.path.exists(self.config.unzip_dir):
                os.makedirs(self.config.unzip_dir)
            request.urlretrieve(self.config.source_URL, target_file)
            logger.info(f"File : {filename} has been downloaded")
            with zipfile.ZipFile(target_file, "r") as zip_ref: